This is an expansion of example_optimize_simple.ipynb

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import pandas as pd
import seaborn as sns

from delayed_reactant_labeling.predict import DRL
from delayed_reactant_labeling.optimize import RateConstantOptimizerTemplate
from delayed_reactant_labeling.visualize import VisualizeMultipleSolutions

In [ ]:
reactions = [
    ('k1', ['A', 'cat'], ['B'],),
    ('k-1', ['B'], ['A', 'cat'],),
    ('k2', ['B'], ['C', 'cat']),

    # labeled
    ('k1', ['A-d10', 'cat'], ['B-d10'],),
    ('k-1', ['B-d10'], ['A-d10', 'cat'],),
    ('k2', ['B-d10'], ['C-d10', 'cat'])
]
concentration_initial = {'A': 1, 'cat': 1 / 5}
concentration_labeled = {'A-d10': 1}
dilution_factor = 1
time_pre = np.linspace(0, 10, 50)
time_post = np.linspace(10, 90, 8 * 50)
rate_values = [0.01, 0.1, 1, 5]  # the model will try these values

In [ ]:
def explore_boundary(k1, kr1, k2, runs):
    path = f'optimization/example_model_boundaries/k1_{k1}_kr1_{kr1}_k2_{k2}/'
    os.mkdir(path)

    #"real" fake data
    rate_constants_real = {'k1': k1, 'k-1': kr1, 'k2': k2}
    drl_real = DRL(rate_constants=rate_constants_real, reactions=reactions)
    real_data_pre, real_data = drl_real.predict_concentration(
        t_eval_pre=time_pre,
        t_eval_post=time_post,
        dilution_factor=dilution_factor,
        initial_concentrations=concentration_initial,
        labeled_concentration=concentration_labeled)
    fig, axs = plt.subplots(1, 2, sharey='row', figsize=(10, 4), layout='tight', width_ratios=(1, 5))
    real_data_pre.to_pandas().plot('time', ax=axs[0])
    real_data.to_pandas().plot('time', ax=axs[1])
    fig.savefig(f'{path}/real_data.png', dpi=500)

    real_data = real_data.with_columns([
        pl.when(pl.col(col) < 1e-8)
            .then(1e-8)
            .otherwise(pl.col(col))
            .alias(col)
        for col in real_data.columns
    ])
    print('min is:', real_data.to_numpy().min())

    class RateConstantOptimizer(RateConstantOptimizerTemplate):
        @staticmethod
        def create_prediction(x: np.ndarray, x_description: list[str]) -> pl.DataFrame:
            rate_constants = pd.Series(x, x_description)
            drl = DRL(reactions=reactions, rate_constants=rate_constants)
            _, pred_labeled = drl.predict_concentration(
                t_eval_pre=time_pre,
                t_eval_post=time_post,
                initial_concentrations=concentration_initial,
                labeled_concentration=concentration_labeled,
                dilution_factor=dilution_factor,
                rtol=1e-8,
                atol=1e-8, )
            return pred_labeled

        @staticmethod
        def calculate_curves(data: pl.DataFrame) -> dict[str, pl.Series]:
            curves = {}
            for chemical in ['A', 'B', 'C']:
                chemical_sum = data[[chemical, f'{chemical}-d10']].sum(axis=1)
                curves[f'ratio_{chemical}'] = data[chemical] / chemical_sum
            return curves

    def METRIC(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        return np.average(np.abs(y_pred - y_true), axis=0)

    RCO = RateConstantOptimizer(raw_weights={}, experimental=real_data, metric=METRIC)

    dimension_description = ['k1', 'k-1', 'k2']
    bounds = [(1e-9, 100),    # k1
              (0,    100),    # k-1 / kr1 as input to the func.
              (1e-9, 100),]   # k2
    RCO.optimize_multiple(path=f'{path}/multiple_guess/', n_runs=runs, x_bounds=bounds, x_description=dimension_description, n_jobs=-2, maxiter=1000)

In [ ]:
# TODO look at labeled cat
# TODO look at implemented TIC
# TODO uniform vs loguniform
for _k1 in [0.1]:
    for _kr1 in rate_values:
        for _k2 in rate_values:
            try:
                explore_boundary(_k1, _kr1, _k2, 100)
            except Exception as e:
                print(f"{_k1}, {_kr1}, {_k2} had exception: {e}")

In [78]:
def analyze_boundary(k1, kr1, k2):
    path = f'optimization/example_model_boundaries/k1_{k1}_kr1_{kr1}_k2_{k2}/'

    #"real" fake data
    rate_constants_real = {'k1': k1, 'k-1': kr1, 'k2': k2}
    drl_real = DRL(rate_constants=rate_constants_real, reactions=reactions)
    real_data_pre, real_data = drl_real.predict_concentration(
        t_eval_pre=time_pre,
        t_eval_post=time_post,
        dilution_factor=dilution_factor,
        initial_concentrations=concentration_initial,
        labeled_concentration=concentration_labeled)

    # add noise
    rng = np.random.default_rng(42)
    fake_data = []
    for col in real_data.columns[:-1]:  # last column contains time array
        noise_dynamic = real_data[col]**0.5 * rng.normal(loc=0, scale=0.04, size=(real_data.shape[0]))  # S/N increases with sqrt(I)
        noise_static = rng.normal(loc=0, scale=0.005, size=(real_data.shape[0]))
        fake_col = real_data[col] + noise_dynamic + noise_static  # noise is loosely based on intensity
        fake_col[fake_col < 1e-10] = 1e-10  # no negative intensity
        fake_data.append(fake_col)

    fake_data.append(real_data['time'])
    fake_data = pl.DataFrame(fake_data, real_data.columns)

    class RateConstantOptimizer(RateConstantOptimizerTemplate):
        @staticmethod
        def create_prediction(x: np.ndarray, x_description: list[str]) -> pl.DataFrame:
            rate_constants = pd.Series(x, x_description)
            drl = DRL(reactions=reactions, rate_constants=rate_constants)
            _, pred_labeled = drl.predict_concentration(
                t_eval_pre=time_pre,
                t_eval_post=time_post,
                initial_concentrations=concentration_initial,
                labeled_concentration=concentration_labeled,
                dilution_factor=dilution_factor,
                rtol=1e-8,
                atol=1e-8, )
            return pred_labeled

        @staticmethod
        def calculate_curves(data: pl.DataFrame) -> dict[str, pl.Series]:
            curves = {}
            for chemical in ['A', 'B', 'C']:
                chemical_sum = data[[chemical, f'{chemical}-d10']].sum(axis=1)
                curves[f'ratio_{chemical}'] = data[chemical] / chemical_sum
            return curves

    def METRIC(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        return np.average(np.abs(y_pred - y_true), axis=0)

    RCO = RateConstantOptimizer(raw_weights={}, experimental=fake_data, metric=METRIC)

    VMS = VisualizeMultipleSolutions(path=f'{path}/multiple_guess/')

    sorted_errors = VMS.complete_found_error[VMS.complete_found_error.argsort()]
    best_rates = VMS.complete_optimal_X[VMS.complete_found_error.argmin()]
    plateau_max = sorted_errors[0] + 10 * (sorted_errors[sorted_errors != sorted_errors[0]][0] - sorted_errors[0])
    plateau_runs = VMS.complete_found_error < plateau_max

    plateau_size = sum(plateau_runs)
    plateau_error = VMS.complete_found_error[plateau_runs].mean()
    plateau_std = VMS.complete_found_error[plateau_runs].std()
    platea_rates = VMS.complete_optimal_X[plateau_runs].mean(axis=0)

    pred = RCO.create_prediction(x=[k1, kr1, k2], x_description=['k1', 'k-1', 'k2'])
    real_error = RCO.calculate_error_functions(pred)
    real_error_weighed = RCO.calculate_total_error(real_error)

    results = pd.Series({
        "k1": k1,
        "kr1": kr1,
        "k2": k2,
        "A_td": pred["A"][40:60].mean(),
        "B_td": pred["B"][40:60].mean(),
        "C_td": pred["C"][40:60].mean(),
        "real_error": real_error_weighed,
        "best_error": sorted_errors[0],
        "best_rate_k1": best_rates[0] / k1,
        "best_rate_kr1": best_rates[1] / kr1,
        "best_rate_k2": best_rates[2] / k2,
        "best_rate_k1_kr1_ratio": (best_rates[0] / best_rates[1]) / ( k1 / kr1),
        "plateau_max": plateau_max,
        "plateau_size": plateau_size,
        "plateau_error": plateau_error,
        "plateau_std": plateau_std,
        "plateau_rate_k1": platea_rates[0] / k1,
        "plateau_rate_kr1": platea_rates[1] / kr1,
        "plateau_rate_k2": platea_rates[2] / k2,
        "plateau_rate_k1_kr1_ratio": (platea_rates[0] / platea_rates[1]) / ( k1 / kr1),
    })
    return results

In [79]:
data = []
for _k1 in [0.1]:
    for _kr1 in rate_values:
        for _k2 in rate_values:
            try:
                data.append(analyze_boundary(_k1, _kr1, _k2))
            except Exception as e:
                print(f"{_k1}, {_kr1}, {_k2} had exception: {e}")

100it [00:01, 58.31it/s]
100it [00:01, 64.34it/s]
100it [00:01, 65.76it/s]
100it [00:01, 68.83it/s]
100it [00:01, 68.10it/s]
100it [00:01, 70.19it/s]
100it [00:01, 63.37it/s]
100it [00:01, 69.46it/s]
100it [00:01, 73.58it/s]
100it [00:01, 71.03it/s]
100it [00:01, 69.74it/s]
100it [00:01, 67.03it/s]
100it [00:01, 68.40it/s]
100it [00:01, 69.36it/s]
100it [00:01, 65.47it/s]
100it [00:01, 67.75it/s]
C:\Users\mdingemans\AppData\Local\Temp\ipykernel_9228\3580393658.py:83: RuntimeWarning: divide by zero encountered in scalar divide
  "best_rate_k1_kr1_ratio": (best_rates[0] / best_rates[1]) / ( k1 / kr1),


In [80]:
df = pd.DataFrame(data)
df

,k1,kr1,k2,A_td,B_td,C_td,real_error,best_error,best_rate_k1,best_rate_kr1,best_rate_k2,best_rate_k1_kr1_ratio,plateau_max,plateau_size,plateau_error,plateau_std,plateau_rate_k1,plateau_rate_kr1,plateau_rate_k2,plateau_rate_k1_kr1_ratio
0,0.1,0.01,0.01,0.852300,0.128643,0.019057,0.131409,2.756360e-05,0.999736,1.000400,0.999308,0.999336,1.583406e-04,3.0,4.034043e-05,1.031161e-05,0.999954,0.998765,1.000875,1.001190
1,0.1,0.01,0.10,0.800974,0.070200,0.128826,0.095065,7.093300e-06,1.000351,1.002324,0.999872,0.998032,1.715230e-05,7.0,1.119296e-05,2.509735e-06,1.000156,1.000854,0.999982,0.999303
2,0.1,0.01,1.00,0.703064,0.012228,0.284708,0.265939,2.420120e-05,0.989939,0.065932,1.007729,15.014630,4.116320e-05,15.0,2.659812e-05,2.543073e-06,0.990998,0.160474,1.006758,6.175437
3,0.1,0.01,5.00,0.678631,0.002640,0.318729,0.345486,9.254000e-07,0.998112,0.070878,1.001449,14.082134,1.528400e-06,7.0,9.782429e-07,2.160283e-08,0.997988,0.010126,1.001579,98.557331
4,0.1,0.10,0.01,0.912955,0.073858,0.013187,0.169692,6.122000e-06,1.000199,1.000200,0.999480,0.999999,2.934100e-05,6.0,1.142757e-05,4.306504e-06,1.000090,0.999980,1.000632,1.000110
5,0.1,0.10,0.10,0.852609,0.049294,0.098097,0.108879,3.069700e-06,1.000151,1.000170,0.999873,0.999981,3.932700e-06,2.0,3.112850e-06,4.315000e-08,1.000099,1.000117,0.999952,0.999983
6,0.1,0.10,1.00,0.721257,0.011598,0.267145,0.270515,3.846890e-05,0.983394,0.841036,1.012537,1.169264,1.520284e-03,12.0,2.105653e-04,5.327558e-05,0.910819,0.088833,1.074417,10.253182
7,0.1,0.10,5.00,0.683163,0.002611,0.314226,0.345322,9.561000e-06,0.980051,0.000207,1.015976,4726.233512,9.581000e-06,6.0,9.563300e-06,1.252996e-09,0.980047,0.000035,1.015985,28357.299551
8,0.1,1.00,0.01,0.980487,0.016259,0.003255,0.395408,4.051000e-07,0.999991,0.999985,1.000314,1.000006,2.901100e-06,2.0,5.299000e-07,1.248000e-07,1.000051,1.000001,1.000018,1.000050
9,0.1,1.00,0.10,0.955246,0.014795,0.029959,0.227129,4.220000e-07,1.000107,1.000016,0.999907,1.000091,6.130000e-07,2.0,4.315500e-07,9.550000e-09,1.000118,1.000031,0.999912,1.000087


In [ ]:
    VMS = VisualizeMultipleSolutions(f'{path}/multiple_guess/', max_guess=500)

    # error / run
    fig, ax = VMS.show_error_all_runs()
    ax.set_ylabel("error")
    eq = VMS.complete_found_error < 1.005 * VMS.complete_found_error.min()
    ax_ins = ax.inset_axes([0.15, 0.5, 0.4, 0.4])
    ax_ins.scatter(np.arange(sum(eq)), sorted(VMS.complete_found_error[eq]))
    ax.indicate_inset_zoom(ax_ins, edgecolor='black')
    ax.set_title(f"Error using real rate constants: {base_error:.4f}")
    fig.savefig(f'{path}/error_per_run.png', dpi=500)
    plt.close(fig)

    # k values for best runs
    fig, axs = plt.subplots(3, 1, layout='tight', figsize=(8, 6))
    for i in range(3):
        ax = axs[i]
        eq = np.where(VMS.complete_found_error < VMS.complete_found_error.min()*1.005)
        best_X = VMS.complete_optimal_X[eq]
        sns.histplot(best_X[:, i], ax=ax)
        yl, yu = ax.get_ylim()
        k = list(rate_constants_real.values())[i]
        ax.plot([k, k], [yl, yu], label='true', color="tab:orange")
        ax.set_ylim(yl, yu)
        ax.set_title(VMS.x_description[i])
    axs[0].legend()
    fig.savefig(f'{path}/best_ks.png', dpi=500)
    plt.close(fig)

    fig, ax = VMS.show_rate_constants(max_error=VMS.complete_found_error.min()*1.01, index_constant_values=None)
    ax.set_yscale("linear")
    ax.scatter([1, 2, 3], list(rate_constants_real.values()), label="true")
    ax.legend()
    fig.savefig(f'{path}/rate_constants_boxplot.png', dpi=500)
    plt.close(fig)